# Summarizing data

Pandas has a number of functions for summarizing data.

Those with a database background will probably be most. comfortable with the GroupBy method and the agg method. 

Those who are more used to working with spreadsheets might be most comfortable with the pivot_table method.

Either way, Pandas has you covered, although there is a lot of overlap between the two.

Lets read in the data we created in the lecture on intersections

In [ ]:
%matplotlib inline
import geopandas as gpd

raptor_buffer = gpd.read_file('data/intersections.gpkg', layer = 'raptor_buffer')
raptor_linear = gpd.read_file('data/intersections.gpkg', layer = 'raptor_linear')

In [ ]:
raptor_buffer.sort_values('Nest_ID')

With this data we can ask questions like "What projects are impacted by Nest 68?"

In [ ]:
raptor_buffer[raptor_buffer['Nest_ID']==68]

## Aggregate functions 

Or "How many acres of ROW are impacted by Nest 68?"

In [ ]:
raptor_buffer[raptor_buffer['Nest_ID']==68].sum()['area_ha']

## Challenge!

How many meters of the linear Project 1107 are impacted by Raptor nests?

In [ ]:
raptor_linear[raptor_linear['Project']==1107]['length_intersection'].sum()

You can also use other aggregate functions like count(), mean(), std(), min(), max(), etc.  A full list can be found at [Panda's aggregate functions](https://cmdlinetips.com/2019/10/pandas-groupby-13-functions-to-aggregate/)

In [ ]:
raptor_linear[raptor_linear['Project']==1107]['length_intersection'].mean()

You can also use the describe method to get a full set of aggregate functions

In [ ]:
raptor_linear[raptor_linear['Project']==1107]['length_intersection'].describe()

## The groupby method

The group by method in Pandas allows you to summarize a set of data agregated over one or more groups.

The basic syntax is a column name or set of column names and an aggregate function as follows

In [ ]:
raptor_buffer.groupby(['Project']).sum()

Note that Project is now a named index.  It doesn't make much sense to sum the *Nest_ID* or *length_m* fields.  But it is interesting to see the sum of each project that is impacted by raptor nests.  Lets look at just the *area_ha* column sorted by area.

In [ ]:
raptor_buffer.groupby(['Project']).sum()['area_ha'].sort_values()

We might also be interested in seeing the number of nests impacted by each project

In [ ]:
raptor_buffer.groupby(['Project']).count()['area_ha'].sort_values()

Or in how many projects are impacted by each nest

In [ ]:
raptor_buffer.groupby(['Nest_ID']).count()['area_ha'].sort_values()

If you want more detail we can add a second level of grouping.  For instance to see how many acreas of each project are impacted by each species of raptor.

In [ ]:
raptor_buffer.groupby(['Project', 'recentspec']).sum()['area_ha']

And we can go even further to group by nest status just by adding a third grouping column

In [ ]:
raptor_buffer.groupby(['Project', 'recentspec', 'recentstat']).count()['area_ha']

At this level of detail however it might make more sense to set the first level of grouping to a broader category like project type

In [ ]:
raptor_buffer.groupby(['type', 'recentspec', 'recentstat']).count()['area_ha']

## The agg method

Can be applied to any dataframe.

Allows you to specify exactly which aggregate functions to display for each column.

You can provide a list of aggregate functions which will be applied to each numeric column.

In [ ]:
raptor_buffer.groupby(['type', 'recentspec', 'recentstat']).agg(['count', 'sum'])['area_ha']

Or, if you want more fine-scale control of what is included,  you can provide a dictionary mapping columns to a list of aggregate functions for that column.

In [ ]:
raptor_buffer.groupby(['type', 'recentspec', 'recentstat']).agg({"Nest_ID":['count'], "area_ha":['sum','mean','std']})

## Challenge #2

How many electric lines have swainson hawk nests within 333 meters and how many nests are affected by each electric line

In [ ]:
raptor_buffer[raptor_buffer['type']=='Electric Line'][raptor_buffer['recentspec']=='Swainsons Hawk'].groupby(['Project', 'recentspec']).agg({"Nest_ID":['count']})